# Reducing to a single dataframe

In this notebook, we will go through our dataset to create a single dataframe to apply some of our machine learning models.  We've already cleaned much of the data.  Our target variables are loan status (via categorization) and loan amount (via regression), so we wil

In [1]:
import pandas as pd
import numpy as np
from financial_data import *
from misc import *

In [2]:
account, card, client, disp, district, loan, order, trans = load_clean_dataset()

In [3]:
loan.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status
0,4959,2,1994-01-05,0.129680,0.25,0.319488,A
1,4961,19,1996-04-29,0.043179,0.00,0.231001,B
2,4962,25,1997-12-08,0.043179,0.00,0.231001,A
3,4967,37,1998-10-14,0.535129,1.00,0.520924,D
4,4968,38,1998-04-19,0.180520,0.75,0.208516,C


In [4]:
account.head()

,account_id,district_id,frequency,account_date
0,1,18,monthly,1995-03-24
1,2,1,monthly,1993-02-26
2,3,5,monthly,1997-07-07
3,4,12,monthly,1996-02-21
4,5,15,monthly,1997-05-30


In [5]:
target_var = loan['status']

Our target variable for this categorization is loan status, therefore we will use this as our base variable.  We will therefore begin with the load table.  The loan id is irrelevant and not joined on any other table, so we will drop it.  However, we will need to join the account table

In [6]:
loan_account = loan.join(account.set_index('account_id'), on = 'account_id', how = 'left')

In [7]:
loan_account.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,district_id,frequency,account_date
0,4959,2,1994-01-05,0.129680,0.25,0.319488,A,1,monthly,1993-02-26
1,4961,19,1996-04-29,0.043179,0.00,0.231001,B,21,monthly,1995-04-07
2,4962,25,1997-12-08,0.043179,0.00,0.231001,A,68,monthly,1996-07-28
3,4967,37,1998-10-14,0.535129,1.00,0.520924,D,20,monthly,1997-08-18
4,4968,38,1998-04-19,0.180520,0.75,0.208516,C,19,weekly,1997-08-08


In [8]:
loan_account = loan_account.join(district.set_index('district_id'), on = 'district_id', how = 'left')

In [9]:
loan_account.columns

Index(['loan_id', 'account_id', 'loan_date', 'loan_amount', 'duration',
       'payments', 'status', 'district_id', 'frequency', 'account_date',
       'district_name', 'region', 'no_of_inhibitants',
       'no_of_municipalities_with_inhabitants_less_than_499',
       'no_of_municipalities_with_inhabitants_between_500_to_1999',
       'no_of_municipalities_with_inhabitants_between_1000_to_9999',
       'no_of_municipalities_with_inhabitants_greater_than_10000',
       'no_of_cities', 'ratio_of_urban_inhabitants', 'average_salary',
       'unemployment_rate_95', 'unemployment_rate_96',
       'no_of_enterpreneurs_per_1000_inhabitants', 'no_of_commited_crimes_95',
       'no_of_commited_crimes_96'],
      dtype='object')

In order to save some items, we will have to make calculations for number of and types of cards, therefore we will have to calculate the number of cards per account and number of dispondents per account.  Using value_counts() we know that no dispondent has more than one card, so we can simply join dispondent on the card dataframe

In [10]:
dispcard = card.join(disp.set_index('disp_id'), on = 'disp_id', how = 'outer')

In [11]:
card.head()

,card_id,disp_id,card_type,issued_date
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [12]:
dispcard = pd.merge(disp, card, on = 'disp_id', how = 'left')

In [13]:
dispcard.head(10)

,disp_id,client_id,account_id,disp_type,card_id,card_type,issued_date
0,1,1,1,owner,NaN,NaN,NaT
1,2,2,2,owner,NaN,NaN,NaT
2,3,3,2,user,NaN,NaN,NaT
3,4,4,3,owner,NaN,NaN,NaT
4,5,5,3,user,NaN,NaN,NaT
5,6,6,4,owner,NaN,NaN,NaT
6,7,7,5,owner,NaN,NaN,NaT
7,8,8,6,owner,NaN,NaN,NaT
8,9,9,7,owner,1.0,gold,1998-10-16
9,10,10,8,owner,NaN,NaN,NaT


In [14]:
dispcard.groupby('account_id')['disp_id'].agg('count').value_counts()

1    3631
2     869
Name: disp_id, dtype: int64

In [15]:
account = account.set_index('account_id')

In [16]:
account['clients'] = dispcard.groupby('account_id')['disp_id'].agg('count')

In [17]:
account['num_cards']=dispcard.groupby('account_id')['card_id'].agg('count')

In [18]:
account['card_issued'] = dispcard.groupby('account_id')['issued_date'].min()

In [30]:
owners = disp[disp['disp_type']=='owner']

In [31]:
owners = owners.join(client.set_index('client_id'), on='client_id', how = 'left')

In [72]:
owners.head()

,disp_id,client_id,account_id,disp_type,gender,birth_date,district_id
0,1,1,1,owner,F,1970-12-13,18
1,2,2,2,owner,M,1945-02-04,1
3,4,4,3,owner,M,1956-12-01,5
5,6,6,4,owner,M,1919-09-22,12
6,7,7,5,owner,M,1929-01-25,15


In [38]:
account['owner_gender'] = owners.groupby('account_id')['gender'].max()

In [44]:
account['owner_birth'] = owners.groupby('account_id')['birth_date'].max()

In [73]:
trans.head()

,trans_id,account_id,trans_date,trans_type,trans_operation,trans_amount,trans_balance,trans_k_symbol
0,1,1,1995-03-24,credit,credit in cash,0.011442,0.167991,unknown
1,5,1,1995-04-13,credit,from another bank,0.042094,0.182663,unknown
2,6,1,1995-05-13,credit,from another bank,0.042094,0.247656,unknown
3,7,1,1995-06-13,credit,from another bank,0.042094,0.271017,unknown
4,8,1,1995-07-13,credit,from another bank,0.042094,0.285293,unknown


In [55]:
credits = trans[trans['trans_type']=='credit']

In [64]:
withdrawals = trans[trans['trans_type']=='withdrawal']

In [74]:
account['num_credits']=credits.groupby('account_id')['trans_amount'].agg('count')

In [75]:
account['total_credits']=credits.groupby('account_id')['trans_amount'].sum()

In [77]:
account['num_withdrawals']=withdrawals.groupby('account_id')['trans_amount'].agg('count')

In [78]:
account['withdrawal_total']=withdrawals.groupby('account_id')['trans_amount'].sum()

In [79]:
account.head()

,district_id,frequency,account_date,clients,num_cards,card_issued,owner_gender,owner_birth,num_credits,total_credits,num_withdrawals,withdrawal_total
account_id,,,,,,,,,,,,
1,18,monthly,1995-03-24,1,0,NaT,F,1970-12-13,102,2.223364,137,2.069451
2,1,monthly,1993-02-26,2,0,NaT,M,1945-02-04,152,18.272941,326,17.785572
3,5,monthly,1997-07-07,2,0,NaT,M,1956-12-01,46,1.980114,71,1.395515
4,12,monthly,1996-02-21,1,0,NaT,M,1919-09-22,65,2.200789,121,1.815069
5,15,monthly,1997-05-30,1,0,NaT,M,1929-01-25,35,1.115435,49,0.794073


In [80]:
order.head()

,order_id,account_id,order_amount,order_k_symbol
0,29401,1,0.164707,household
1,29402,2,0.226578,loan
2,29403,2,0.488206,household
3,29404,3,0.076205,household
4,29405,3,0.021907,unknown


In [97]:
account['num_orders'] = order.groupby('account_id')['order_amount'].agg('count')

In [98]:
account['order_total'] = order.groupby('account_id')['order_amount'].sum()

In [99]:
account.head()

,district_id,frequency,account_date,clients,num_cards,card_issued,owner_gender,owner_birth,num_credits,total_credits,num_withdrawals,withdrawal_total,num_orders,order_total
account_id,,,,,,,,,,,,,,
1,18,monthly,1995-03-24,1,0,1900-01-01,F,1970-12-13,102,2.223364,137,2.069451,1.0,0.164707
2,1,monthly,1993-02-26,2,0,1900-01-01,M,1945-02-04,152,18.272941,326,17.785572,2.0,0.714784
3,5,monthly,1997-07-07,2,0,1900-01-01,M,1956-12-01,46,1.980114,71,1.395515,3.0,0.335865
4,12,monthly,1996-02-21,1,0,1900-01-01,M,1919-09-22,65,2.200789,121,1.815069,2.0,0.225858
5,15,monthly,1997-05-30,1,0,1900-01-01,M,1929-01-25,35,1.115435,49,0.794073,1.0,0.179222


In [92]:
account['card_issued'].fillna(np.datetime64('1900-01-01'), inplace = True)

In [100]:
loan_account = loan.join(account, on = 'account_id', how = 'left')

In [102]:
loan_account.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,district_id,frequency,account_date,...,num_cards,card_issued,owner_gender,owner_birth,num_credits,total_credits,num_withdrawals,withdrawal_total,num_orders,order_total
0,4959,2,1994-01-05,0.129680,0.25,0.319488,A,1,monthly,1993-02-26,...,0,1900-01-01,M,1945-02-04,152,18.272941,326,17.785572,2.0,0.714784
1,4961,19,1996-04-29,0.043179,0.00,0.231001,B,21,monthly,1995-04-07,...,0,1900-01-01,F,1939-04-23,152,9.075469,151,8.951213,1.0,0.169491
2,4962,25,1997-12-08,0.043179,0.00,0.231001,A,68,monthly,1996-07-28,...,0,1900-01-01,M,1962-02-09,82,17.098112,192,16.752643,4.0,0.713003
3,4967,37,1998-10-14,0.535129,1.00,0.520924,D,20,monthly,1997-08-18,...,0,1900-01-01,M,1952-08-26,36,5.686831,94,5.161670,4.0,0.692259
4,4968,38,1998-04-19,0.180520,0.75,0.208516,C,19,weekly,1997-08-08,...,0,1900-01-01,F,1940-01-30,48,3.527071,82,3.017002,4.0,0.649338


In [104]:
combined = loan_account.join(district.set_index('district_id'), on = 'district_id', how = 'left')

In [106]:
combined.to_csv('combined_data.csv', index = False)

In [107]:
loan.shape

(682, 7)